# Imports and Data

## Import Libraries

In [ ]:
!pip install affine6p
import affine6p

In [ ]:
import os
import numpy as np
import tensorflow as tf
import random

from numpy.random import randint

import tifffile as tiff
from osgeo import gdal

import cv2 as cv
from IPython.display import Image

from keras.models import Model
from keras.layers import Input, Activation, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.optimizers import Adam

from skimage import transform, filters

## Reshape Pictures
No need to execute, if pictures are already reshaped to (224, 224, 3)


In [ ]:
def resize_images():
    j = [20, 22, 24]
    for trajectory in j:
        i = 1
        while i <= 50:
            path_read = "drive/MyDrive/MLDM/cropped/" + str(trajectory) + "/cropped_" + str(i) + ".jpg"
            img = cv.imread(path_read)
            resized_image = cv.resize(img, (224, 224))
            path_write = "drive/MyDrive/MLDM/cropped/"+ str(trajectory) +"/cropped_" + str(i) + ".jpg"
            cv.imwrite(path_write, resized_image, )
            i += 1

Overwrite cropped_reshaped folder with cropped images

In [ ]:
def overwrite_reshaped_images():
    j = [20, 22, 24]
    for trajectory in j:
        i = 1
        while i <= 50:
            path_read = "drive/MyDrive/MLDM/cropped/" + str(trajectory) + "/cropped_" + str(i) + ".jpg"
            img = cv.imread(path_read)
            path_write = "drive/MyDrive/MLDM/cropped_reshaped/" + str(trajectory) + "/cropped_" + str(i) + ".jpg"
            cv.imwrite(path_write, img)
            i += 1

Reshape pictures to (224, 224, 4)

In [ ]:
def reshape_images():
    j = [20, 22, 24]
    for trajectory in j:
        i = 1
        while i <= 50:
            path_read = "drive/MyDrive/MLDM/cropped_reshaped/" + str(trajectory) + "/cropped_" + str(i) + ".jpg"
            img = cv.imread(path_read)
            #print(img.shape)
            #reshaped_image = np.concatenate((img, np.zeros_like(img[..., :1])), axis=-1)                # -> does the job!
            img = cv.cvtColor(img, cv.COLOR_RGB2RGBA)
            #print(img)
            #print(img.shape)
            #reshaped_image = np.dstack((img,, np.zeros((224, 224, 1), dtype=np.uint8)))                # -> dstack adds something in an existing axis
            #reshaped_image = np.stack((img, axis=4, np.zeros((224, 224), dtype=np.uint8)))             # -> stack should add an axis but doesn't
            #reshaped_image = np.expand_dims(img, axis=3)                                               # -> deletes all the images somehow
            path_write = "drive/MyDrive/MLDM/cropped_reshaped/" + str(trajectory) + "/cropped_" + str(i) + ".png"
            cv.imwrite(path_write, img)
            i += 1

In [ ]:
#resize_images()

In [ ]:
#overwrite_reshaped_images()

In [ ]:
#reshape_images()

# Feature-based registration of historical aerial images by Area Minimization

## First Step:
#### Digitize free-form lines in stable areas: Find areas where region has not changed significantly over time - Time-Invariant line features (TIL)


## Second Step:
#### Register the free-form lines to recover unknown Exterior Orientation (EO) parameter


## Third Step:
#### Area Minimization: Determine EO by minimizing the area formed between corresponding free-form-lines

# Co-registration of multitemporal UAV image datasets for monitoring applications

## First Step:
#### Images blocks from input epochs

## Second Step:
#### Add saved EOPs from Epoch 2 to the corresponding reference images in the block

## Third Step:
#### Treat Camera interior orientation

## Fourth Step:
#### Bundle Block Adjustment between reference and input images

# Change Detection in VHR Imagery with severe co-registration errors using Deep Learning

**Code of the paper**: https://github.com/vkristoll/change-detection-autoencoder/blob/main/coregister.py

**STANet**: https://github.com/justchenhao/STANet

## Review

### Define Autoencoder

In [ ]:
#Create function to define the autoencoder model
def autoencoder():
    #Define the patch size
    original_img_size = (224, 224, 4)

    #encoder
    input_img = Input(shape=original_img_size)
    x = Conv2D(64, (3, 3), padding='same')(input_img)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), padding='same')(x)
    encoded = Activation('relu')(x)

    #decoder
    x = Conv2DTranspose(32, (4,4), strides=(2,2), padding='same')(encoded)
    x = Activation('relu')(x)
    x = Conv2DTranspose(64, (4,4), strides=(2,2), padding='same')(x)
    x = Activation('relu')(x)
    x= Conv2D(4, (3, 3), padding='same')(x)
    decoded = Activation('sigmoid')(x)

    #Define the model
    model = Model(input_img, decoded)

    #Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

#Define the autoencoder model
model=autoencoder()
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 4)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      2368      
                                                                 
 activation (Activation)     (None, 224, 224, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 32)      18464     
                                                                 
 activation_1 (Activation)   (None, 112, 112, 32)      0         
                                                             

### Create change maps

In [ ]:
#Load the weights
#The autoencoder model is  defined in "autoencoder_training.py"
model.load_weights("/content/drive/MyDrive/MLDM/cropped_resized/new_weights.h5")

#Run the line below to find the indexes of the layers
layer_names = [layer.name for layer in model.layers]

#Define submodels that calculate the model predictions for the selected indexes
model1 = Model(inputs=model.inputs, outputs=model.layers[3].output)
model2 = Model(inputs=model.inputs, outputs=model.layers[6].output)
model3 = Model(inputs=model.inputs, outputs=model.layers[10].output)
model4 = Model(inputs=model.inputs, outputs=model.layers[12].output)

#Define function that resizes the multilevel feature maps and concatenates them
def feat(x_train):

    feat1 = model1.predict(x_train)
    feat2 = model2.predict(x_train)
    feat3 = model3.predict(x_train)
    feat4 = model4.predict(x_train)

    x1 = tf.image.resize(feat1,[128,128])
    x2 = tf.image.resize(feat2,[128,128])
    x3 = tf.image.resize(feat3,[128,128])
    x4 = tf.image.resize(feat4,[128,128])

    F = tf.concat([x2,x1,x4,x3],3)
    return F

In [ ]:
#Define paths to 2 folders that contain the patches of the 1st and 2nd date (224x224x4 px)
#Create sorted list of the files
dir1_path="/content/drive/MyDrive/MLDM/cropped_cropped/cropped_cropped/24/"
dir1_list=os.listdir(dir1_path)
dir1_sorted_list=sorted(dir1_list)

dir2_path="/content/drive/MyDrive/MLDM/cropped_cropped/cropped_cropped/24/"
dir2_list=os.listdir(dir2_path)
dir2_sorted_list=sorted(dir2_list)

#Define path to folder to store the change maps
dir3_path="/content/drive/MyDrive/MLDM/cropped_cropped/result_24/"

#Count number of files
number_files=len(dir1_list)

In [ ]:
def generate_changemap(i, j, x):
    #Access the first date folder
    os.chdir(dir1_path)
    #Read the first date image
    im1 = gdal.Open(dir1_sorted_list[i])
    im1=np.array(im1.ReadAsArray())
    #Change the order of the axes to width,height,channels
    im1=im1.transpose(1,2,0)
    im1 = np.concatenate((im1, np.zeros_like(im1[..., :1])), axis=-1)
    #Convert values to [0,1]
    im1=im1/255
    #Expand the dimensions so that the model can read it (batch_size=1)
    im1=np.expand_dims(im1,0)

    #Store the file name and define name of the change map
    if x == True:
        filename=os.path.basename("changemap_big" + str(i) + "_" + str(j) + ".tiff")
    else:
        filename=os.path.basename("changemap" + str(i) + "_" + str(j) + ".tiff")
    print(filename)

    os.chdir(dir2_path)
    im2 = gdal.Open(dir2_sorted_list[j])
    im2=np.array(im2.ReadAsArray())
    im2=im2.transpose(1,2,0)
    im2 = np.concatenate((im2, np.zeros_like(im2[..., :1])), axis=-1)
    im2= im2/255
    im2=np.expand_dims(im2,0)

    F1=feat(im1) #Features from image patch 1
    #Calculate the square value of the feature maps
    F1=tf.square(F1)
    F2=feat(im2) #Features from image patch 2
    F2=tf.square(F2)
    #Subtract the feature maps from the 2 dates
    d=tf.subtract(F1,F2)
    d=tf.square(d)
    #Create the change map by summing values from the feature maps
    d=tf.reduce_sum(d,axis=3)

    #Resize the change map
    dis=d.numpy()
    dis=dis.transpose(1,2,0)
    dis = tf.image.resize(dis,[224,224], method="nearest")
    dis=dis.numpy()
    dis=np.resize(dis,[224,224])

    #Save the change map
    os.chdir(dir3_path)
    tiff.imsave(filename,dis)

In [ ]:
for i in range(0, number_files-1, 1):
    generate_changemap(i, i+1, False)

changemap0_1.tiff
1/1 [==============================] - 0s 16ms/step
changemap1_2.tiff
1/1 [==============================] - 0s 18ms/step


<ipython-input-72-240fe6bb57ec>:50: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(filename,dis)


1/1 [==============================] - 0s 20ms/step
changemap2_3.tiff
1/1 [==============================] - 0s 21ms/step
changemap3_4.tiff
1/1 [==============================] - 0s 16ms/step
changemap4_5.tiff
1/1 [==============================] - 0s 18ms/step
changemap5_6.tiff
1/1 [==============================] - 0s 17ms/step
changemap6_7.tiff
1/1 [==============================] - 0s 19ms/step
changemap7_8.tiff
1/1 [==============================] - 0s 17ms/step
changemap8_9.tiff
1/1 [==============================] - 0s 18ms/step
changemap9_10.tiff
1/1 [==============================] - 0s 17ms/step
changemap10_11.tiff
1/1 [==============================] - 0s 20ms/step
changemap11_12.tiff
1/1 [==============================] - 0s 19ms/step
changemap12_13.tiff
1/1 [==============================] - 0s 28ms/step
changemap13_14.tiff
1/1 [==============================] - 0s 16ms/step
changemap14_15.tiff
1/1 [==============================] - 0s 31ms/step
changemap15_16.tiff
1/1 [==

### Otsu segmentation

In [ ]:
#Define the folder path that contains the change maps with the float values
#Create list of the files
dir1_path="/content/drive/MyDrive/MLDM/cropped_resized/result_20/"
dir1_list=os.listdir(dir1_path)
#Define folder path to store the binary change maps
dir2_path="/content/drive/MyDrive/MLDM/cropped_resized/result_20_png/"

#Count the number of the files
number_files=len(dir1_list)

#Iterate over the files and create the binary change maps
for i in range (number_files):

    #Print the index of the file
    print(" The file index is %s" %i)

    #Access the directory of the change maps (float)
    os.chdir(dir1_path)
    #Read image
    im1 = gdal.Open(dir1_list[i])
    im1=np.array(im1.ReadAsArray())

    #Store the file name and define the name of the binary change map
    filename=os.path.basename(dir1_list[i])
    filename2=filename.split('.')[0] + ".png"

    #Access the directory of the binary change maps
    os.chdir(dir2_path)

    #Calculate the Otsu threshold
    """
    if im1.min() == 0 and im1.max()==0:
        val=0
    else:
        val = filters.threshold_otsu(im1[:,:])

    #Create array to store the binary values
    A=np.uint8(np.zeros((224,224)))

    for i1 in range(224):
        for i2 in range(224):
            if im1[i1,i2]>val:
                A[i1,i2]=255
            else:
                A[i1,i2]=0
    """
    cv.imwrite(filename2, im1)

 The file index is 0
 The file index is 1
 The file index is 2
 The file index is 3
 The file index is 4
 The file index is 5
 The file index is 6
 The file index is 7
 The file index is 8
 The file index is 9
 The file index is 10
 The file index is 11
 The file index is 12
 The file index is 13
 The file index is 14
 The file index is 15
 The file index is 16
 The file index is 17
 The file index is 18
 The file index is 19
 The file index is 20
 The file index is 21
 The file index is 22
 The file index is 23
 The file index is 24
 The file index is 25
 The file index is 26
 The file index is 27
 The file index is 28
 The file index is 29
 The file index is 30
 The file index is 31
 The file index is 32
 The file index is 33
 The file index is 34
 The file index is 35
 The file index is 36
 The file index is 37
 The file index is 38
 The file index is 39
 The file index is 40
 The file index is 41
 The file index is 42
 The file index is 43
 The file index is 44
 The file index is 4

### Train Autoencoder

In [ ]:
#Creating training data

#Import libraries
import os
import numpy as np
from osgeo import gdal

'''Define path to 3 folders that contain first date patches (224x224x4 px) for the 4 study areas (Granada, Rhodes, Venice, Tønsberg).
The original 16 bit images have been previously converted to 8 bit. They contain R, G, B, NIR bands'''

#Create list with the names of the patches
dir1_path="/content/drive/MyDrive/MLDM/cropped_cropped/cropped_cropped/20/"
dir1_list=os.listdir(dir1_path)

dir2_path="/content/drive/MyDrive/MLDM/cropped_cropped/cropped_cropped/22/"
dir2_list=os.listdir(dir2_path)

dir3_path="/content/drive/MyDrive/MLDM/cropped_cropped/cropped_cropped/24/"
dir3_list=os.listdir(dir3_path)

#Create list that contains the paths of the 4 folders
dir_path_list=[dir1_path,dir2_path,dir3_path]

#Create list that contains the 4 lists of the names of the patches
dir_sorted_list=[sorted(dir1_list),sorted(dir2_list), sorted(dir3_list)]
print(dir_sorted_list)
#Create list that stores the number of files (patches) in each folder
number_files_list=[len(dir1_list),len(dir2_list),len(dir3_list)]

#Create list that contains float arrays to store the patches for each study area

train_list=[np.float16(np.zeros((len(dir1_list),224,224,4))),np.float16(np.zeros((len(dir2_list),224,224,4))),
            np.float16(np.zeros((len(dir3_list),224,224,4)))]

[['cropped1.jpg', 'cropped10.jpg', 'cropped11.jpg', 'cropped12.jpg', 'cropped13.jpg', 'cropped14.jpg', 'cropped15.jpg', 'cropped16.jpg', 'cropped17.jpg', 'cropped18.jpg', 'cropped19.jpg', 'cropped2.jpg', 'cropped20.jpg', 'cropped21.jpg', 'cropped22.jpg', 'cropped23.jpg', 'cropped24.jpg', 'cropped25.jpg', 'cropped26.jpg', 'cropped27.jpg', 'cropped28.jpg', 'cropped29.jpg', 'cropped3.jpg', 'cropped30.jpg', 'cropped31.jpg', 'cropped32.jpg', 'cropped33.jpg', 'cropped34.jpg', 'cropped35.jpg', 'cropped36.jpg', 'cropped37.jpg', 'cropped38.jpg', 'cropped39.jpg', 'cropped4.jpg', 'cropped40.jpg', 'cropped41.jpg', 'cropped42.jpg', 'cropped43.jpg', 'cropped44.jpg', 'cropped45.jpg', 'cropped46.jpg', 'cropped47.jpg', 'cropped48.jpg', 'cropped49.jpg', 'cropped5.jpg', 'cropped50.jpg', 'cropped6.jpg', 'cropped7.jpg', 'cropped8.jpg', 'cropped9.jpg'], ['cropped1.jpg', 'cropped10.jpg', 'cropped11.jpg', 'cropped12.jpg', 'cropped13.jpg', 'cropped14.jpg', 'cropped15.jpg', 'cropped16.jpg', 'cropped17.jpg', 'cr

In [ ]:
#Add the patches to the train_list
for i in range (3):
    os.chdir(dir_path_list[i])
    for k in range(0, number_files_list[i],1):
        #Print the index of the patch
        print(f"x_train {i}: k is {k}")
        #Read the patch
        x_1 = gdal.Open(dir_sorted_list[i][k])
        #Convert it to an array
        x_1 = np.array(x_1.ReadAsArray())
        #Change the order of the axes to width,height,channels
        x_1 = x_1.transpose(1,2,0)
        #Reshape
        x_1 = np.concatenate((x_1, np.zeros_like(x_1[..., :1])), axis=-1)
        #Convert values to [0,1]
        x_1 = x_1/255
        print(x_1.shape)
        #Add the patch to the train_list
        train_list[i][k,:,:,:]=x_1

x_train 0: k is 0
(224, 224, 4)
x_train 0: k is 1
(224, 224, 4)
x_train 0: k is 2
(224, 224, 4)
x_train 0: k is 3
(224, 224, 4)
x_train 0: k is 4
(224, 224, 4)
x_train 0: k is 5
(224, 224, 4)
x_train 0: k is 6
(224, 224, 4)
x_train 0: k is 7
(224, 224, 4)
x_train 0: k is 8
(224, 224, 4)
x_train 0: k is 9
(224, 224, 4)
x_train 0: k is 10
(224, 224, 4)
x_train 0: k is 11
(224, 224, 4)
x_train 0: k is 12
(224, 224, 4)
x_train 0: k is 13
(224, 224, 4)
x_train 0: k is 14
(224, 224, 4)
x_train 0: k is 15
(224, 224, 4)
x_train 0: k is 16
(224, 224, 4)
x_train 0: k is 17
(224, 224, 4)
x_train 0: k is 18
(224, 224, 4)
x_train 0: k is 19
(224, 224, 4)
x_train 0: k is 20
(224, 224, 4)
x_train 0: k is 21
(224, 224, 4)
x_train 0: k is 22
(224, 224, 4)
x_train 0: k is 23
(224, 224, 4)
x_train 0: k is 24
(224, 224, 4)
x_train 0: k is 25
(224, 224, 4)
x_train 0: k is 26
(224, 224, 4)
x_train 0: k is 27
(224, 224, 4)
x_train 0: k is 28
(224, 224, 4)
x_train 0: k is 29
(224, 224, 4)
x_train 0: k is 30
(

In [ ]:
def batch_generator():

    #Define batch size
    batch_size=8
    #Select randomly one of the 3 study areas
    randomindexim=randint(3)

    #Select random sample of indexes equal to the batch size from the randomly selected study area
    if randomindexim==0:
       randomindexsample=random.sample(range(0,len(dir1_list)),batch_size)

    elif  randomindexim==1:
       randomindexsample=random.sample(range(0,len(dir2_list)),batch_size)

    elif  randomindexim==2:
       randomindexsample=random.sample(range(0,len(dir3_list)),batch_size)

    #Create array with patches equal to the batch size
    X=np.float16(np.zeros((batch_size,224,224,4)))
    c=0
    for i in range(1, batch_size, 1):
        c=c+1
        X[c-1,:,:,:]=train_list[randomindexim][randomindexsample[i],:,:,:]

    # Return training batch. It's the same for the input and the output because the model is an autoencoder.
    return X,X

In [ ]:
#Define the number of epochs
epochs=400

#Define the number of training steps
#dir1_list, dir2_list, dir3_list, dir4_list are defined in "create_training_data.py"
train_steps=int((len(dir1_list)+ len(dir2_list) + len(dir3_list))/14)

for i in range (epochs):
    for j in range (train_steps):
        #batch_generator is defined in "training_batch_generator.py"
        X,y = batch_generator()
        loss=model.train_on_batch(X,y)

        print('>%d, %d/%d, d=%.3f' % (i+1, j+1, train_steps, loss))

#Save the weights
model.save_weights("/content/drive/MyDrive/MLDM/cropped_cropped/new_weights.h5")

>1, 1/10, d=0.093
>1, 2/10, d=0.099
>1, 3/10, d=0.167
>1, 4/10, d=0.096
>1, 5/10, d=0.130
>1, 6/10, d=0.131
>1, 7/10, d=0.084
>1, 8/10, d=0.126
>1, 9/10, d=0.115
>1, 10/10, d=0.091
>2, 1/10, d=0.078
>2, 2/10, d=0.054
>2, 3/10, d=0.078
>2, 4/10, d=0.077
>2, 5/10, d=0.079
>2, 6/10, d=0.075
>2, 7/10, d=0.063
>2, 8/10, d=0.048
>2, 9/10, d=0.065
>2, 10/10, d=0.080
>3, 1/10, d=0.083
>3, 2/10, d=0.063
>3, 3/10, d=0.081
>3, 4/10, d=0.062
>3, 5/10, d=0.040
>3, 6/10, d=0.039
>3, 7/10, d=0.038
>3, 8/10, d=0.039
>3, 9/10, d=0.070
>3, 10/10, d=0.036
>4, 1/10, d=0.059
>4, 2/10, d=0.064
>4, 3/10, d=0.037
>4, 4/10, d=0.034
>4, 5/10, d=0.032
>4, 6/10, d=0.036
>4, 7/10, d=0.034
>4, 8/10, d=0.056
>4, 9/10, d=0.071
>4, 10/10, d=0.065
>5, 1/10, d=0.076
>5, 2/10, d=0.062
>5, 3/10, d=0.053
>5, 4/10, d=0.064
>5, 5/10, d=0.083
>5, 6/10, d=0.066
>5, 7/10, d=0.060
>5, 8/10, d=0.065
>5, 9/10, d=0.056
>5, 10/10, d=0.064
>6, 1/10, d=0.030
>6, 2/10, d=0.030
>6, 3/10, d=0.047
>6, 4/10, d=0.065
>6, 5/10, d=0.061
>6, 6

Get Model weights

In [ ]:
#The autoencoder model is  defined in "autoencoder_training.py"
model.load_weights("/content/drive/MyDrive/MLDM/cropped_cropped/new_weights.h5")

np.set_printoptions(threshold=np.inf)
with open('/content/drive/MyDrive/MLDM/cropped_cropped/new_weights.txt', 'w') as f:
    #for layer in model.layers: print(layer.get_config(), layer.get_weights())
    for layer in model.layers:
       f.write(str(layer.get_config()))
       f.write(str(layer.get_weights()))


# STANet

**STANet Code**: https://github.com/justchenhao/STANet

Test Demo

In [ ]:
"""This module contains simple helper functions """
from __future__ import print_function
import torch
import numpy as np
from PIL import Image
import os
import ntpath


def save_images(images, img_dir, name):
    """save images in img_dir, with name
    iamges: torch.float, B*C*H*W
    img_dir: str
    name: list [str]
    """
    for i, image in enumerate(images):
        print(image.shape)
        image_numpy = tensor2im(image.unsqueeze(0),normalize=False)*255
        basename = os.path.basename(name[i])
        print('name:', basename)
        save_path = os.path.join(img_dir,basename)
        save_image(image_numpy,save_path)


def save_visuals(visuals,img_dir,name):
    """
    """
    name = ntpath.basename(name)
    name = name.split(".")[0]
    print(name)
    # save images to the disk
    for label, image in visuals.items():
        image_numpy = tensor2im(image)
        img_path = os.path.join(img_dir, '%s_%s.png' % (name, label))
        save_image(image_numpy, img_path)


def tensor2im(input_image, imtype=np.uint8, normalize=True):
    """"Converts a Tensor array into a numpy image array.

    Parameters:
        input_image (tensor) --  the input image tensor array
        imtype (type)        --  the desired type of the converted numpy array
    """
    if not isinstance(input_image, np.ndarray):
        if isinstance(input_image, torch.Tensor):  # get the data from a variable
            image_tensor = input_image.data
        else:
            return input_image
        image_numpy = image_tensor[0].cpu().float().numpy()  # convert it into a numpy array
        if image_numpy.shape[0] == 1:  # grayscale to RGB
            image_numpy = np.tile(image_numpy, (3, 1, 1))

        image_numpy = np.transpose(image_numpy, (1, 2, 0))
        if normalize:
            image_numpy = (image_numpy + 1) / 2.0 * 255.0  # post-processing: tranpose and scaling

    else:  # if it is a numpy array, do nothing
        image_numpy = input_image
    return image_numpy.astype(imtype)


def save_image(image_numpy, image_path):
    """Save a numpy image to the disk

    Parameters:
        image_numpy (numpy array) -- input numpy array
        image_path (str)          -- the path of the image
    """
    image_pil = Image.fromarray(image_numpy)
    image_pil.save(image_path)


def print_numpy(x, val=True, shp=False):
    """Print the mean, min, max, median, std, and size of a numpy array

    Parameters:
        val (bool) -- if print the values of the numpy array
        shp (bool) -- if print the shape of the numpy array
    """
    x = x.astype(np.float64)
    if shp:
        print('shape,', x.shape)
    if val:
        x = x.flatten()
        print('mean = %3.3f, min = %3.3f, max = %3.3f, median = %3.3f, std=%3.3f' % (
            np.mean(x), np.min(x), np.max(x), np.median(x), np.std(x)))


def mkdirs(paths):
    """create empty directories if they don't exist

    Parameters:
        paths (str list) -- a list of directory paths
    """
    if isinstance(paths, list) and not isinstance(paths, str):
        for path in paths:
            mkdir(path)
    else:
        mkdir(paths)


def mkdir(path):
    """create a single empty directory if it didn't exist

    Parameters:
        path (str) -- a single directory path
    """
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
import argparse
import os
import torch
import models
import data


class BaseOptions():
    """This class defines options used during both training and test time.

    It also implements several helper functions such as parsing, printing, and saving the options.
    It also gathers additional options defined in <modify_commandline_options> functions in both dataset class and model class.
    """

    def __init__(self):
        """Reset the class; indicates the class hasn't been initailized"""
        self.initialized = False

    def initialize(self, parser):
        """Define the common options that are used in both training and test."""
        # basic parameters
        parser.add_argument('--dataroot', type=str, default='./LEVIR-CD', help='path to images (should have subfolders A, B, label)')
        parser.add_argument('--val_dataroot', type=str, default='./LEVIR-CD', help='path to images in the val phase (should have subfolders A, B, label)')

        parser.add_argument('--name', type=str, default='experiment_name', help='name of the experiment. It decides where to store samples and models')
        parser.add_argument('--gpu_ids', type=str, default='0', help='gpu ids: e.g. 0  0,1,2, 0,2. use -1 for CPU')
        parser.add_argument('--checkpoints_dir', type=str, default='./checkpoints', help='models are saved here')
        # model parameters
        parser.add_argument('--model', type=str, default='CDF0', help='chooses which model to use. [CDF0 | CDFA]')
        parser.add_argument('--input_nc', type=int, default=3, help='# of input image channels: 3 for RGB ')
        parser.add_argument('--output_nc', type=int, default=3, help='# of output image channels: 3 for RGB')
        parser.add_argument('--arch', type=str, default='mynet3', help='feature extractor architecture | mynet3')
        parser.add_argument('--f_c', type=int, default=64, help='feature extractor channel num')
        parser.add_argument('--n_class', type=int, default=2, help='# of output pred channels: 2 for num of classes')
        parser.add_argument('--init_type', type=str, default='normal', help='network initialization [normal | xavier | kaiming | orthogonal]')
        parser.add_argument('--init_gain', type=float, default=0.02, help='scaling factor for normal, xavier and orthogonal.')
        parser.add_argument('--SA_mode', type=str, default='BAM', help='choose self attention mode for change detection, | ori |1 | 2 |pyramid, ...')
        # dataset parameters
        parser.add_argument('--dataset_mode', type=str, default='changedetection', help='chooses how datasets are loaded. [changedetection | concat | list | json]')
        parser.add_argument('--val_dataset_mode', type=str, default='changedetection', help='chooses how datasets are loaded. [changedetection | concat| list | json]')
        parser.add_argument('--dataset_type', type=str, default='CD_LEVIR', help='chooses which datasets too load. [LEVIR | WHU ]')
        parser.add_argument('--val_dataset_type', type=str, default='CD_LEVIR', help='chooses which datasets too load. [LEVIR | WHU ]')
        parser.add_argument('--split', type=str, default='train', help='chooses wihch list-file to open when use listDataset. [train | val | test]')
        parser.add_argument('--val_split', type=str, default='val', help='chooses wihch list-file to open when use listDataset. [train | val | test]')
        parser.add_argument('--json_name', type=str, default='train_val_test', help='input the json name which contain the file names of images of different phase')
        parser.add_argument('--val_json_name', type=str, default='train_val_test', help='input the json name which contain the file names of images of different phase')
        parser.add_argument('--ds', type=int, default='1', help='self attention module downsample rate')
        parser.add_argument('--angle', type=int, default=0, help='rotate angle')
        parser.add_argument('--istest', type=bool, default=False, help='True for the case without label')

        parser.add_argument('--serial_batches', action='store_true', help='if true, takes images in order to make batches, otherwise takes them randomly')
        parser.add_argument('--num_threads', default=4, type=int, help='# threads for loading data')
        parser.add_argument('--batch_size', type=int, default=1, help='input batch size')
        parser.add_argument('--load_size', type=int, default=286, help='scale images to this size')
        parser.add_argument('--crop_size', type=int, default=256, help='then crop to this size')
        parser.add_argument('--max_dataset_size', type=int, default=float("inf"), help='Maximum number of samples allowed per dataset. If the dataset directory contains more than max_dataset_size, only a subset is loaded.')
        parser.add_argument('--preprocess', type=str, default='resize_and_crop', help='scaling and cropping of images at load time [resize_and_crop | none]')
        parser.add_argument('--no_flip', type=bool, default=True, help='if specified, do not flip(left-right) the images for data augmentation')

        parser.add_argument('--display_winsize', type=int, default=256, help='display window size for both visdom and HTML')
        # additional parameters
        parser.add_argument('--epoch', type=str, default='latest', help='which epoch to load? set to latest to use latest cached model')
        parser.add_argument('--load_iter', type=int, default='0', help='which iteration to load? if load_iter > 0, the code will load models by iter_[load_iter]; otherwise, the code will load models by [epoch]')
        parser.add_argument('--verbose', action='store_true', help='if specified, print more debugging information')
        parser.add_argument('--suffix', default='', type=str, help='customized suffix: opt.name = opt.name + suffix: e.g., {model}_{netG}_size{load_size}')
        self.initialized = True
        return parser

    def gather_options(self):
        """Initialize our parser with basic options(only once).
        Add additional model-specific and dataset-specific options.
        These options are defined in the <modify_commandline_options> function
        in model and dataset classes.
        """
        if not self.initialized:  # check if it has been initialized
            parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
            parser = self.initialize(parser)

        # get the basic options
        opt, _ = parser.parse_known_args()

        # modify model-related parser options
        model_name = opt.model
        model_option_setter = models.get_option_setter(model_name)
        parser = model_option_setter(parser, self.isTrain)
        opt, _ = parser.parse_known_args()  # parse again with new defaults

        # modify dataset-related parser options
        dataset_name = opt.dataset_mode
        if dataset_name != 'concat':
            dataset_option_setter = data.get_option_setter(dataset_name)
            parser = dataset_option_setter(parser, self.isTrain)

        # save and return the parser
        self.parser = parser
        return parser.parse_args()

    def print_options(self, opt):
        """Print and save options

        It will print both current options and default values(if different).
        It will save options into a text file / [checkpoints_dir] / opt.txt
        """
        message = ''
        message += '----------------- Options ---------------\n'
        for k, v in sorted(vars(opt).items()):
            comment = ''
            default = self.parser.get_default(k)
            if v != default:
                comment = '\t[default: %s]' % str(default)
            message += '{:>25}: {:<30}{}\n'.format(str(k), str(v), comment)
        message += '----------------- End -------------------'
        print(message)

        # save to the disk
        expr_dir = os.path.join(opt.checkpoints_dir, opt.name)
        util.mkdirs(expr_dir)
        file_name = os.path.join(expr_dir, 'opt.txt')
        with open(file_name, 'wt') as opt_file:
            opt_file.write(message)
            opt_file.write('\n')

    def parse(self):
        """Parse our options, create checkpoints directory suffix, and set up gpu device."""
        opt = self.gather_options()
        opt.isTrain = self.isTrain   # train or test

        # process opt.suffix
        if opt.suffix:
            suffix = ('_' + opt.suffix.format(**vars(opt))) if opt.suffix != '' else ''
            opt.name = opt.name + suffix

        self.print_options(opt)

        # set gpu ids
        str_ids = opt.gpu_ids.split(',')
        opt.gpu_ids = []
        for str_id in str_ids:
            id = int(str_id)
            if id >= 0:
                opt.gpu_ids.append(id)
        if len(opt.gpu_ids) > 0:
            torch.cuda.set_device(opt.gpu_ids[0])

        self.opt = opt
        return self.opt

ModuleNotFoundError: ignored

In [ ]:
class TestOptions(BaseOptions):
    """This class includes test options.

    It also includes shared options defined in BaseOptions.
    """

    def initialize(self, parser):
        parser = BaseOptions.initialize(self, parser)  # define shared options
        parser.add_argument('--ntest', type=int, default=float("inf"), help='# of test examples.')
        parser.add_argument('--results_dir', type=str, default='./results/', help='saves results here.')
        parser.add_argument('--aspect_ratio', type=float, default=1.0, help='aspect ratio of result images')
        parser.add_argument('--phase', type=str, default='test', help='train, val, test, etc')
        # Dropout and Batchnorm has different behavioir during training and test.
        parser.add_argument('--eval', action='store_true', help='use eval mode during test time.')
        parser.add_argument('--num_test', type=int, default=50, help='how many test images to run')
        # To avoid cropping, the load_size should be the same as crop_size
        parser.set_defaults(load_size=parser.get_default('crop_size'))
        self.isTrain = False
        return parser

In [ ]:
import time
from options.test_options import TestOptions
from data import create_dataset
from models import create_model
import os
from util.util import save_images
import numpy as np
from util.util import mkdir
from PIL import Image


def make_val_opt(opt):

    # hard-code some parameters for test
    opt.num_threads = 0   # test code only supports num_threads = 1
    opt.batch_size = 1    # test code only supports batch_size = 1
    opt.serial_batches = True  # disable data shuffling; comment this line if results on randomly chosen images are needed.
    opt.no_flip = True    # no flip; comment this line if results on flipped images are needed.
    opt.no_flip2 = True    # no flip; comment this line if results on flipped images are needed.

    opt.display_id = -1   # no visdom display; the test code saves the results to a HTML file.
    opt.phase = 'val'
    opt.preprocess = 'none1'
    opt.isTrain = False
    opt.aspect_ratio = 1
    opt.eval = True

    return opt



def val(opt):

    dataset = create_dataset(opt)  # create a dataset given opt.dataset_mode and other options
    model = create_model(opt)      # create a model given opt.model and other options
    model.setup(opt)               # regular setup: load and print networks; create schedulers
    save_path = opt.results_dir
    mkdir(save_path)
    model.eval()

    for i, data in enumerate(dataset):
        if i >= opt.num_test:  # only apply our model to opt.num_test images.
            break
        model.set_input(data)  # unpack data from data loader
        pred = model.test(val=False)           # run inference return pred

        img_path = model.get_image_paths()     # get image paths
        if i % 5 == 0:  # save images to an HTML file
            print('processing (%04d)-th image... %s' % (i, img_path))

        save_images(pred, save_path, img_path)


def pred_image(data_root, results_dir):
    opt = TestOptions().parse()   # get training options
    opt = make_val_opt(opt)
    opt.phase = 'test'
    opt.dataset_mode = 'changedetection'
    opt.n_class = 2
    opt.SA_mode = 'PAM'
    opt.arch = 'mynet3'
    opt.model = 'CDFA'
    opt.epoch = 'pam'
    opt.num_test = np.inf
    opt.name = 'pam'
    opt.dataroot = data_root
    opt.results_dir = results_dir

    val(opt)

ModuleNotFoundError: ignored